## Connecting to a DB

In [1]:
from mb_sql.conn import get_engine, get_session, get_base, get_metadata
from mb_sql.sql import read_sql,engine_execute,to_sql
import sqlalchemy as sa

In [2]:
engine = get_engine(name='postgresql' , db= 'postgres', user='postgres' , password= 'postgres', host= 'localhost', port= 5432, echo=False)
engine2 = get_engine(name='postgresql' , db= 'postgres_2', user='postgres' , password= 'postgres', host= 'localhost', port= 5432, echo=False)
session = get_session(engine)
base = get_base()
metadata= get_metadata(base,engine)

## Executing query functions in an engine

In [15]:
a = engine_execute(engine,'CREATE TABLE IF NOT EXISTS test3 (id serial PRIMARY KEY, num integer, data varchar);')
a_2 = engine_execute(engine2,'CREATE TABLE IF NOT EXISTS test3 (id serial PRIMARY KEY, num integer, data varchar);')

In [4]:
#b = engine_execute(conn,"INSERT INTO test3(id,num, data) VALUES (1,100, 'abc')")

In [5]:
c = engine_execute(engine,"SELECT * FROM test3;")
d = engine_execute(engine,"SELECT * FROM test2;")
print(c.fetchall())
print(d.fetchall())

[(1, 100, 'abc')]
[('dave', 12), ('lave', 32), ('read', 54)]


In [3]:
a_sql = read_sql('SELECT * FROM test1;',engine)
a_sql

,id,values
0,1,a
1,2,b
2,3,c


In [13]:
a_sql2 = a_sql.copy()
to_sql(a_sql2,engine,'test4',schema='public',if_exists='replace',index=False)

Tue 14:17:28,563 INF DataFrame written to test4 table.                                                                               


In [39]:
q1 ="SELECT schema_name FROM information_schema.schemata WHERE schema_name NOT IN ('information_schema', 'mysql', 'performance_schema') ORDER BY schema_name;"
read_sql(q1,engine)

Tue 15:57:17,622 INF SQL query executed successfully.                                                                                


,schema_name
0,pg_catalog
1,pg_temp_1
2,pg_toast
3,pg_toast_temp_1
4,public
5,testing_schema


## using extra functions

In [3]:
from mb_sql.utils import list_schemas,list_tables,rename_table,drop_table,drop_schema,create_schema,create_index,clone_db
from mb.utils import logger

In [4]:
list_schemas(engine)

,schema_name
0,pg_catalog
1,pg_temp_1
2,pg_toast
3,pg_toast_temp_1
4,public
5,testing_schema


In [5]:
list_tables(engine,schema='public')

,table_name
0,test1
1,test2
2,test3
3,test4_renamed2


In [6]:
rename_table('test4_renamed','test4_renamed2',engine,logger=logger)

Thu 01:47:33,991 INF Table test4_renamed2 renamed to test4_renamed.                                                                  


In [7]:
list_tables(engine,schema='public')

,table_name
0,test1
1,test2
2,test3
3,test4_renamed


In [8]:
drop_table('test4_renamed',engine,logger=logger)

Thu 01:47:58,465 INF Table test4_renamed dropped.                                                                                    


In [9]:
list_tables(engine,schema='public')

,table_name
0,test1
1,test2
2,test3


In [10]:
create_schema('test_schema2',engine,logger=logger)

Thu 01:48:31,762 INF Schema test_schema2 created.                                                                                    


In [11]:
list_schemas(engine)

,schema_name
0,pg_catalog
1,pg_temp_1
2,pg_toast
3,pg_toast_temp_1
4,public
5,testing_schema
6,test_schema2


In [12]:
drop_schema('test_schema2',engine,logger=logger)

Thu 01:48:43,417 INF Schema test_schema2 dropped.                                                                                    


In [14]:
list_schemas(engine)

,schema_name
0,pg_catalog
1,pg_temp_1
2,pg_toast
3,pg_toast_temp_1
4,public
5,testing_schema


In [16]:
a = engine_execute(engine,'CREATE TABLE IF NOT EXISTS test_last_update (id serial PRIMARY KEY, num integer, data varchar,last_updated timestamp);')


In [17]:
a = '''INSERT INTO test_last_update (num, data, last_updated) VALUES
(1, 'foo', '2023-07-19 10:23:45'),
(2, 'bar', '2023-07-20 09:34:21'),
(3, 'baz', '2023-07-20 14:12:18'),
(4, 'qux', '2023-07-19 15:45:12'),
(5, 'quux', '2023-07-20 11:23:54'),
(6, 'corge', '2023-07-19 17:09:37'),
(7, 'grault', '2023-07-20 13:56:42'),
(8, 'garply', '2023-07-19 22:18:09'),
(9, 'waldo', '2023-07-20 08:47:33'),
(10, 'fred', '2023-07-19 16:37:28'),
(11, 'plugh', '2023-07-20 12:09:57'),
(12, 'xyzzy', '2023-07-19 19:22:13'),
(13, 'thud', '2023-07-20 10:01:04'),
(14, 'foo', '2023-07-19 14:55:39'),
(15, 'bar', '2023-07-20 15:27:16');'''

engine_execute(engine,a)


In [19]:
a = ''' CREATE TABLE IF NOT EXISTS test_last_update_2 (
  id serial PRIMARY KEY,
  num integer,
  data varchar,
  last_updated timestamp
);

INSERT INTO test_last_update_2 (num, data, last_updated) VALUES
  (1, 'foo', '2023-07-23 10:23:45'),
  (2, 'bar', '2023-07-24 09:34:21'),
  (3, 'baz', '2023-07-25 14:12:18'),
  (4, 'qux', '2023-07-26 15:45:12'),
  (5, 'quux', '2023-07-27 11:23:54'),
  (6, 'corge', '2023-07-28 17:09:37'),
  (7, 'grault', '2023-07-29 13:56:42'),
  (8, 'garply', '2023-07-30 22:18:09'),
  (9, 'waldo', '2023-08-01 08:47:33'),
  (10, 'fred', '2023-08-02 16:37:28'),
  (11, 'plugh', '2023-08-03 12:09:57'),
  (12, 'xyzzy', '2023-08-04 19:22:13'),
  (13, 'thud', '2023-08-05 10:01:04'),
  (14, 'foo', '2023-08-06 14:55:39'),
  (15, 'bar', '2023-08-07 15:27:16'); '''

engine_execute(engine,a)

In [21]:
from mb_sql.update import get_last_updated_timestamp

In [31]:
import pandas as pd
def get_last_updated_timestamp(engine,table_name: str,schema : str ="public",updated_col: str = "updated") -> pd.Timestamp:
    """
    Gets the last updated timestamp from all records of an sqlite table containing the 'updated' field.

    Args:
        engine (sqlalchemy.engine.base.Engine): Engine object.
        table_name (str): tables containing the 'updated' field. 
        schema (str): Name of the schema. Default: None
        updated_col (str): name of the 'updated' field. Default: 'updated'
    Returns:
        ts (pandas.Timestamp): last updated timestamp
    """

    query_str = f"SELECT MAX({updated_col}) AS last_updated FROM {table_name};"
    df = read_sql(query_str,engine)

    ts = pd.Timestamp(df["last_updated"][0])
    if pd.isnull(ts):
        ts = pd.Timestamp("2014-01-01")
    return ts

In [36]:
k = get_last_updated_timestamp(engine,table_name='test_last_update',schema='public',updated_col='last_updated')